## 소개


이 샘플은 환자가 심장병에 걸렸는지를 알아내는 데 사용되는 이진 분류(binary classification) 알고리즘 입니다. 이 예에서는 UCI 저장소에서 가져온 Cleveland Heart Disease 데이터 세트에서 샘플 데이터를 업로드합니다. 데이터 세트는 303개의 개별 데이터로 구성됩니다. 컬럼 설명 및 샘플 데이터는 데이터 저장소를 참조하십시오:
https://archive.ics.uci.edu/ml/datasets/heart+Disease 샘플 데이터를 다운로드하고 데이터를 S3 버킷에 저장한 다음 이 노트북의 셀을 실행하여 자체 모델을 구축하고 배포할 수 있습니다.

이 튜토리얼의 나머지 부분에서는 이진 분류(binary classification) 알고리즘을 사용하여 심장병을 예측하는 방법에 대해 설명합니다



## 사전 설정

### 권한 및 환경변수

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성 및 테스트되었습니다._

먼저 다음을 지정합니다:

- 트레이닝 및 모델 데이터에 사용할 S3 버킷 및 prefix를 지정합니다. S3 리전은 노트북 인스턴스, 트레이닝 및 호스팅과 동일한 리전 내에 있어야 합니다. S3 버킷을 생성했으면 버킷 이름과 prefix를 지정합니다.
- IAM role arn은 데이터에 대한 트레이닝 및 호스팅에 대한 액세스를 제공하는데 사용됩니다. 

In [ ]:
#버킷명 입력
bucket = '{ENTER_BUCKET_NAME}'
prefix = 'sagemaker/heart'

#데이터 파일명 입력 (ex. heart.csv)
data_key = 'heart.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
 
# IAM role 설정
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

### 데이터 수집

 데이터를 수집하기 전에 UCI 저장소의 데이터 세트(예: heart_data.csv)가 S3 버킷에 업로드되었는지 확인하십시오. 기본적으로 SageMaker 역할은 'sageMaker*'로 시작하는 버킷에 액세스할 수 있습니다. 아래 코드는 지정된 S3 버킷에서 데이터를 읽고 샘플을 출력합니다.


In [ ]:
import pandas as pd
import json

# S3로부터 데이터 읽기
heart_data = pd.read_csv(data_location)

#샘플 데이터 출력
heart_data.head()

### 데이터 변환

알고리즘에는 특정 input 및 output 요구사항이 있기 때문에 데이터 세트를 변환하는 것도 데이터 사이언티스트가 학습을 시작하기 전에 거쳐야 하는 프로세스의 일부입니다. Amazon SageMaker의 Linear Learner 알고리즘은 recordIO-wrapped protobuf 포맷을 지원합니다.

아래 코드는 다음을 수행합니다:

데이터를 가져와서 Numpy array로 변환합니다. SageMaker Linear Learner 알고리즘이 기대하는 float32 유형이어야 합니다.

Linear Learner 알고리즘에는 observations를 나타내는 행과 feature의 dimensions 을 나타내는 열이 있는 데이터 매트릭스가 필요합니다. 또한 데이터  요소와 일치하는 라벨이 들어 있는 추가 열이 필요합니다.

input의 경우 모델에 라벨이 지정된 예제(x, y)를 제공합니다. x는 고차원 벡터이고 y는 숫자 라벨입니다. 이진 분류 문제의 경우 라벨은 0 또는 1이어야 합니다.

The Linear Learner 알고리즘은 특성 행렬(features matrix) 과 라벨 벡터(labels vector)가 필요합니다.

라벨 열은 'target' 열입니다. 이 경우 마지막 열을 예측하여 사용자에게 심장병이 있는지 확인하려고 합니다. 값이 1이면 심장병이 있음을 나타내고 0이면 그렇지 않습니다.

In [ ]:
import numpy as np
vectors = np.array(heart_data).astype('float32')

#target 열 - 값은 0 이나 1 이어야 함
labels = vectors[:,13]
print ("label data is")
print (labels)


#target 열을 제거. 이 피쳐는 훈련 데이터의 일부로 사용
training_data = vectors[:, :13]
print ("Training data is")
print (training_data)



S3에 학습 데이터를 업로드해 보겠습니다.

In [ ]:
import io
import os
import sagemaker.amazon.common as smac

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, training_data, labels)
buf.seek(0)

key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))




## 아티팩트 학습
데이터가 학습되면 다음 위치에 업로드됩니다.

In [ ]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

## Linear 모델 학습

데이터를 사전 처리하고 올바른 학습 포맷으로 사용할 수 있게 되면 다음 단계는 데이터를 사용하여 실제로 모델을 훈련하는 것입니다. 이 데이터는 상대적으로 작기 때문에 Linear Learner 학습 알고리즘의 성능을 과시하기 위한 것이 아닙니다.

Amazon SageMaker Python SDK를 사용하여 학습을 시작하고 완료될 때까지 상태를 모니터링합니다. 이 예에서는 약 7분에서 11분 정도 걸립니다. 데이터 세트가 작지만, 하드웨어 프로비저닝 및 알고리즘 컨테이너 로드에 약간의 시간이 소요됩니다.

우리는 이진 분류를 수행하고(환자가 심장 질환을 가지고 있는지 여부), 지정된 컴퓨팅(예: c4.xlarge)에서 모델을 훈련하고, 훈련 세트의 features 또는 dimiensions을 세분화합니다.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker

container = get_image_uri(boto3.Session().region_name, 'linear-learner', "latest")

sess = sagemaker.Session()
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=13,
                           predictor_type='binary_classifier',
                           mini_batch_size=100)

linear.fit({'train': s3_train_data})

## 모델 호스팅
이제 모델을 학습시켰으므로 Amazon SageMaker의 실시간 호스팅 엔드포인트에 모델을 배포할 수 있습니다. 이렇게 하면 모델에서 동적으로 예측(또는 추론)할 수 있습니다.

_참고: Amazon SageMaker를 사용하면 모델 생성 대상이 AWS Lambda, AWS Greengrass, Amazon Redshift, Amazon Athena 또는 기타 다른 곳에서 훈련된 모델을 유연하게 가져올 수 있습니다._

In [ ]:
heartdisease_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

## 모델 검증

마지막으로 모델을 사용할 수 있는지 검증할 수 있습니다. HTTP POST 요청을 엔드포인트로 전달하여 예측을 가져올 수 있습니다. 이 작업을 더 쉽게 하기 위해 Amazon SageMaker Python SDK를 다시 사용하고 알고리즘에 맞는 요청을 직렬화(serialize)하고 응답을 역직렬화(deserialize)하는 방법을 지정합니다.

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

heartdisease_predictor.serializer = csv_serializer
heartdisease_predictor.deserializer = json_deserializer

In [ ]:
print('Endpoint name: {}'.format(heartdisease_predictor.endpoint))

다음 샘플 데이터를 테스트용으로 전달해 보겠습니다. 파일의 단일 레코드입니다.

In [ ]:
vectors[5][0:13]

In [ ]:
result = heartdisease_predictor.predict(vectors[5][0:13])
print(result)

### (옵션) 엔드포인트 삭제

이 노트북에서의 작업이 끝났으며 이후 Cloudformation 템플릿을 통한 배포를 실행하지 않을 시 delete_endpoint 라인을 실행하세요. 실행할 시 생성한 호스팅 엔드포인트가 제거되고 엔드포인트 요금이 부과되지 않습니다.

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(heartdisease_predictor.endpoint)